In [24]:
import cvxpy as cp

In [25]:
# pip install cvxpy

# 저자

- TA: 성준모 (Joonmo Sung)
- SYSTEMS MODELING AND PROGRAMMING LAB DEPARTMENT OF INDUSTRIAL ENGINEERING, YONSEI UNIVERSITY - SYMPLY
- 문의: `sjm21314@naver.com`,`sjm21314@yonsei.ac.kr`
***
본 강의자료는 Keha,A. B.,Khowala,K.,&Fowler,J. W. (2009). Mixed integer programming formulations for single machine scheduling problems. Computers & Industrial Engineering, 56(1), 357-367 논문을 바탕으로 제작되었으며,

반도체데이터사이언스 협동과정 수업 용도 이외의 목적으로 저자의 허락 없이 다른 사람들과 공유할 수 없습니다.

## Dataset
### Set
- N = $\{1,2,...,n\}, j \in N$
### Parameters
- $p_j$: processing time of job $j$
- $d_j$: due date of job $j$
- $w_j$: weight of job $j$
- $r_j$: release date of job $j$

In [26]:
# Offline 상황을 가정. 아래는 임의의 데이터, generation 방법은 후에 설명 예정

n = 5 # 작업의 개수

N = [_ for _ in range(n)] # 작업 set을 리스트 자료구조를 이용해 0, 1, 2,..., n-1 까지 담아줌 [0, 1, 2, 3, 4]
p = [4, 3, 7, 2, 2] # 가공시간 데이터를 리스트 자료구조를 이용해 만들어 줌
d = [5, 6, 8, 8, 17] # 납기 데이터를 리스트 자료구조를 이용해 만들어 줌
w = [5, 4, 3, 2, 1] # 가중치 데이터를 리스트 자료구조를 이용해 만들어 줌
r = [1, 2, 3, 3, 5] # 작업의 release date(도착 시점) 데이터를 리스트 자료구조를 이용해 만들어 줌

M = (sum(p) + sum(r)) * 2 # big M, big M으로 작동할 정도로 크지만, 너무 크면 오류 발생 가능 여기선 최대 완료 시점 * 2로 M 값

In [27]:
obj_select = 'twt' # 'wct', 'tj', 'twt', 'L_max' 중 하나 선택
release_time = True

## 1. Completion time variables

In [28]:
# Decision Variable 만들기 딕셔너리 자료구조를 활용
C = {(j): cp.Variable(nonneg = True) for j in N}
y = {(j, k): cp.Variable(boolean = True) for j in N for k in N if j < k}

In [29]:
C[0]

Variable((), var808, nonneg=True)

### Decision Variables
- $C_j$: 작업 $j$의 시작 시점 결정 $C_j \geq 0 \ \forall j \in N$
- $y_{j,k}$: 작업 $j$가 $k$ 전에 작업 중인지 후에 작업중인지를 결정 $y_{j,k} \in \{0, 1\} \ \forall j,k \in N \ and \ j < k $

### 기본 제약식
- $C_j \geq p_j \quad \forall j \in N$
- $C_j + p_k \leq C_k + M(1-y_{j,k}) \quad \forall j,k \in N, j < k$
- $C_k + p_j \geq C_j + My_{j,k} \quad \forall j,k \in N, j < k$

In [30]:
# 제약식 리스트 선언
constraints = [] # 제약식 담아주는 리스트

# 제약식 추가 += 추가
constraints += [C[j] >= p[j] for j in N]
constraints += [C[j] + p[k] <= C[k] + M*(1-y[j,k]) for j in N for k in N if j < k]
constraints += [C[k] + p[j] <= C[j] + M*y[j,k] for j in N for k in N if j < k]

#### Release time 반영 제약식
$ C_j \geq p_j + r_j \quad \forall j \in N$

In [31]:
# 문제에 release time을 반영하려면 아래 제약식 추가
if release_time == True:
    constraints += [C[j] >= p[j] + r[j] for j in N]

### 목적식

#### To minimize the total weighted completion time
$ Minimize \sum_{j = 1}^n w_j C_j$

In [32]:
if obj_select == 'wct':
    # 목적식
    obj = cp.Minimize(sum(w[j]*C[j] for j in N))

#### To minimize $L_{max}$
$ Minimize \ L_{max}$

$ \\ s.t. \  L_{max} \geq (C_j - d_j) \quad \forall j \in N$

In [33]:
if obj_select == "L_max":
    # Lmax 선언
    L_max = cp.Variable()

    # Lmax 제약식
    constraints += [L_max >= (C[j] - d[j]) for j in N]

    # 목적식
    obj = cp.Minimize(L_max)

#### To minimize the number of tardy jobs

$ Minimize \ \sum_{j=1}^n U_j$

$ \\ s.t. \ C_j \leq d_j + MU_j \quad \forall j \in N$

$ \\ U_j \in \{0, 1\} \quad \forall j \in N$

In [34]:
if obj_select == 'tj':
    U = {(j): cp.Variable(boolean = True) for j in N} # j가 Tardy job이면 1, 아니면 0인 이진변수

    # Tardy job 제약식
    constraints += [C[j] <= d[j] + M*U[j] for j in N]

    # 목적식
    obj = cp.Minimize(sum(U[j] for j in N))

#### To minimize the total weighted tardiness

$ Minimize \ \sum_{j=1}^n w_jT_j$

$ \\ s.t. \ T_j \geq C_j - d_j \quad \forall j \in N$

$ \\ T_j \geq 0 \quad \forall j \in N$

In [35]:
if obj_select == 'twt':
    T = {(j): cp.Variable(nonneg = True) for j in N} # j에 대한 Tardiness 연속형 변수, lower bound 0 (0 이상)

    # Tardiness 제약식
    constraints += [T[j] >= C[j] - d[j] for j in N]

    # 목적식
    obj = cp.Minimize(sum(w[j]*T[j] for j in N))

- 원하는 목적식에 맞게 설정 후 optimize

In [36]:
prob = cp.Problem(obj, constraints)

In [37]:
#(참고) default solver 확인, 설치된 solver 확인
# print(f"Default solver: {prob.solver_stats.solver_name}")
available_solvers = cp.installed_solvers()
print("Available solvers:", available_solvers)

Available solvers: ['CLARABEL', 'CVXOPT', 'GLPK', 'GLPK_MI', 'HIGHS', 'OSQP', 'SCIPY', 'SCS']


In [38]:
prob.solve()
# prob.solve(solver = 'SCIPY')

np.float64(41.0)

In [39]:
print("최적해 상태:", prob.status)
print("최적값:", prob.value)

최적해 상태: optimal
최적값: 41.0


In [40]:
for j, dv in C.items():
    # print(j, dv)
    print(f"C{j+1}: {dv.value}")

C1: 5.0
C2: 8.0
C3: 17.0
C4: 10.0
C5: 19.0


In [41]:
for j, dv in T.items():
    print(f"T{j+1}: {dv.value}")

T1: 0.0
T2: 2.0
T3: 9.0
T4: 2.0
T5: 2.0


In [42]:
for (j, k), dv in y.items():
      print(f"y_{j+1}_{k+1}: {dv.value}")

y_1_2: 1.0
y_1_3: 1.0
y_1_4: 1.0
y_1_5: 1.0
y_2_3: 1.0
y_2_4: 1.0
y_2_5: 1.0
y_3_4: 0.0
y_3_5: 1.0
y_4_5: 1.0
